In [1]:
import pandas as pd
import numpy as np
import pypyodbc as podbc
from ipykernel import kernelapp as app
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
conn = podbc.connect("Driver={SQL Server};Server=CZPHAAASQL16\DM;Database=DataMining2;")
myqr1="SELECT [Id] AS [id], [Make] AS [make], [Model] AS [model], [Makeyear] AS [makeyear], [Tachometer] AS [kilometer], " 
myqr2="[Power] AS [power], [DateIn] AS [datein], [Price] AS [price], [SellerName] AS [seller] "
myqr3="FROM [DataMining2].[dbo].[DmXmls] WHERE (ServerId=1 AND  "
myqr4="DateIn>= Convert(datetime,'2017-01-01') AND DateIn<Convert(datetime,'2017-12-31'));"
myqr = myqr1 + myqr2 + myqr3 + myqr4 
cars = pd.read_sql_query(myqr, conn)
print(len(cars))
print(cars.columns)
cars.head()

400715
Index(['id', 'make', 'model', 'makeyear', 'kilometer', 'power', 'datein',
       'price', 'seller'],
      dtype='object')


,id,make,model,makeyear,kilometer,power,datein,price,seller
0,64878582,Chevrolet,Spark,2008.0,76901.0,38.0,2017-01-01 01:27:43,48000.0,None
1,64878919,Audi,A3,2016.0,3914.0,92.0,2017-01-01 04:07:21,629000.0,"Auto Jarov, s.r.o. - Das WeltAuto"
2,64878920,VW,Golf Sportsvan,2016.0,14852.0,92.0,2017-01-01 04:07:24,468000.0,"Auto Jarov, s.r.o. - Das WeltAuto"
3,64878921,VW,Golf Sportsvan,2016.0,19986.0,92.0,2017-01-01 04:07:27,469000.0,"Auto Jarov, s.r.o. - Das WeltAuto"
4,64878922,Renault,Modus,2008.0,108815.0,55.0,2017-01-01 04:07:31,88000.0,"Auto Jarov, s.r.o. - Das WeltAuto"


In [3]:
len(cars[cars.seller == "Mototechna"])

6970

In [4]:
len(cars[cars.seller == "AAA AUTO"])

51983

In [5]:
len(cars[cars.seller == "AUTO DISKONT"])

8945

In [6]:
cars = cars[(cars.seller!="AAA AUTO")&(cars.seller!="Mototechna")&(cars.seller!="AUTO DISKONT")]
len(cars)

332817

In [7]:
del cars["seller"]

In [8]:
cars.dropna( inplace=True)
calen = len(cars)
print(calen) 

328534


In [9]:
cars.to_csv(path_or_buf='raw_Sauto2017_for_anchors.csv', index=False, index_label=None)
print('done')

done


In [82]:
cars = pd.read_csv("raw_Sauto2017_for_anchors.csv", engine='python', encoding='Latin_1')
calen = len(cars)
print(calen) 
print(cars.columns)
cars.head()

328534
Index(['id', 'make', 'model', 'makeyear', 'kilometer', 'power', 'datein',
       'price'],
      dtype='object')


,id,make,model,makeyear,kilometer,power,datein,price
0,64878582,Chevrolet,Spark,2008.0,76901.0,38.0,2017-01-01 01:27:43,48000.0
1,64878919,Audi,A3,2016.0,3914.0,92.0,2017-01-01 04:07:21,629000.0
2,64878920,VW,Golf Sportsvan,2016.0,14852.0,92.0,2017-01-01 04:07:24,468000.0
3,64878921,VW,Golf Sportsvan,2016.0,19986.0,92.0,2017-01-01 04:07:27,469000.0
4,64878922,Renault,Modus,2008.0,108815.0,55.0,2017-01-01 04:07:31,88000.0


In [83]:
import datetime as dt
import time as tm

cars["year_in"] = cars["datein"].apply(lambda yy: int(yy[0:4]))
cars["month_in"] = cars["datein"].apply(lambda yy: int(yy[5:7]))
cars["day_in"] = cars["datein"].apply(lambda yy: int(yy[8:10]))
cars["day_in"].head()

0    1
1    1
2    1
3    1
4    1
Name: day_in, dtype: int64

In [84]:
cars["days_on"] = 31-cars["day_in"]+30*(12-cars["month_in"])
cars.columns

Index(['id', 'make', 'model', 'makeyear', 'kilometer', 'power', 'datein',
       'price', 'year_in', 'month_in', 'day_in', 'days_on'],
      dtype='object')

In [85]:
cols_to_keep = ["id","make","model","makeyear","kilometer","power","days_on","price"]
cars = cars[cols_to_keep]
cars.head()
#cars = cars.rename(columns={'datein':'days_on'})

,id,make,model,makeyear,kilometer,power,days_on,price
0,64878582,Chevrolet,Spark,2008.0,76901.0,38.0,360,48000.0
1,64878919,Audi,A3,2016.0,3914.0,92.0,360,629000.0
2,64878920,VW,Golf Sportsvan,2016.0,14852.0,92.0,360,468000.0
3,64878921,VW,Golf Sportsvan,2016.0,19986.0,92.0,360,469000.0
4,64878922,Renault,Modus,2008.0,108815.0,55.0,360,88000.0


In [86]:
cars['days_on'].fillna( method="ffill", inplace=True )
cars.sort_values(['days_on'], ascending=True, inplace=True)

In [87]:
cars.head(2)

,id,make,model,makeyear,kilometer,power,days_on,price
116063,75022193,Peugeot,406,1995.0,230000.0,81.0,1,10000.0
77475,75014740,Honda,CRV,2017.0,4.0,88.0,1,535500.0


In [88]:
cars.tail(2)

,id,make,model,makeyear,kilometer,power,days_on,price
83,64886441,VW,Multivan,2009.0,210000.0,96.0,360,412000.0
0,64878582,Chevrolet,Spark,2008.0,76901.0,38.0,360,48000.0


In [89]:
ctk = ["make","model","makeyear","kilometer","power","price"]
cars = cars[ctk]
cars.drop_duplicates( keep='first', inplace=True)
calen = len(cars)
print(calen) 

306261


In [90]:
cti = ["makeyear","kilometer","power","price"]
for i in range(0,4):
    cars[cti[i]] = cars[cti[i]].apply(lambda mt: int(mt))
cars.head()

,make,model,makeyear,kilometer,power,price
116063,Peugeot,406,1995,230000,81,10000
77475,Honda,CRV,2017,4,88,535500
77476,Mazda,626,2003,148500,100,18888
77477,Renault,Laguna,1999,177000,79,29000
77478,Ford,Focus,2005,170000,85,65000


In [91]:
conn=podbc.connect("Driver={SQL Server};Server=CZPHAAADS01;Database=Pricing;uid=data_reader;pwd=data_reader123")
myqr="SELECT [m_m_comb_dm] AS [make_model], [m_m_comb_navi] AS [make_model_trans] FROM [Pricing].[dbo].[trans_make_model];"
trans_mms = pd.read_sql_query(myqr, conn)

In [92]:
cars["make_model"] = cars["make"] + cars["model"] 

In [93]:
cars = pd.merge(cars, trans_mms, on="make_model", how='outer')
print(len(cars)) 

309370


In [94]:
cars = cars.head(306261)
cars.tail()

,make,model,makeyear,kilometer,power,price,make_model,make_model_trans
306256,Dodge,Chalenger,2010.0,126000.0,186.0,590000.0,DodgeChalenger,DodgeChallenger
306257,Proton,413,2001.0,139000.0,55.0,12800.0,Proton413,Proton413
306258,Ligier,JS RC,2015.0,6300.0,4.0,344850.0,LigierJS RC,LigierJS RC
306259,Zastava,1100,1975.0,56119.0,40.0,30250.0,Zastava1100,Zastava1100
306260,Mitsubishi,L400,2001.0,314000.0,64.0,34000.0,MitsubishiL400,MitsubishiL400


In [95]:
cars["make_model_trans"].fillna("0",inplace=True)
missing_trans = cars[cars.make_model_trans=="0"] 
len(missing_trans)

2635

In [96]:
cars.loc[cars.make_model_trans=="0","make_model_trans"] = cars["make_model"]

In [98]:
del cars["make_model"]
cars = cars.rename(columns={"make_model_trans":"make_model"})

In [99]:
cars.dropna( inplace=True)
calen = len(cars)
print(calen) 

306261


In [103]:
cols_to_int = ["makeyear","kilometer","power","price"]
ctilen = len(cols_to_int)
for i in range(0, ctilen):
    cars[cols_to_int[i]] = cars[cols_to_int[i]].apply(lambda ct: int(ct))
cols_to_keep = ["make","model","make_model","makeyear","kilometer","power","price"]  
cars = cars[cols_to_keep]

In [109]:
cars["make_model"] = cars["make_model"].apply(lambda mm: mm.lower()) 

In [110]:
cars.head()

,make,model,make_model,makeyear,kilometer,power,price
0,Peugeot,406,peugeot406,1995,230000,81,10000
1,Peugeot,406,peugeot406,1996,161550,65,27000
2,Peugeot,406,peugeot406,2004,186300,80,44000
3,Peugeot,406,peugeot406,1998,254000,66,13000
4,Peugeot,406,peugeot406,2002,372000,80,16900


In [111]:
cars.to_csv(path_or_buf='Sauto2017_for_anchors_transformed.csv', index=False, index_label=None)
print('done')

done
